In [ ]:
# # run to download mnist dataset
# from mnist import download_mnist
# download_mnist()

/nfs/ghome/live/gren/mambaforge/envs/hsicfuse-env/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
from sampler_mnist import ind_X_Y_corrupt
import numpy as np
import jax
import jax.numpy as jnp
from jax import random
from tqdm.auto import tqdm
from pathlib import Path
Path("results").mkdir(exist_ok=True)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from all_tests import hsicfuse_test, hsic_test
from all_tests import hsicagginc1_test, hsicagginc100_test, hsicagginc200_test, hsicaggincquad_test

### MNIST vary N

In [9]:
corrupt_proportion = 0.75
# rep = 100
# N_values = (200, 400, 600, 800, 1000)
rep = 1
N_values = (200, )

# tests = (hsicagginc1, hsicagginc100, hsicagginc200, hsicaggincquad)
tests = (hsic_test, )
outputs_mnist_n = jnp.zeros((len(tests), len(N_values), rep))
outputs_mnist_n = outputs_hsic_mnist_n.tolist()
key = random.PRNGKey(42)
seed = 42
for i in range(rep):
    for r in range(len(N_values)):
        N = N_values[r]
        key, subkey = random.split(key)
        X, Y = ind_X_Y_corrupt(key, N, corrupt_proportion)
        key, subkey = random.split(key)
        seed += 1
        for j in range(len(tests)):
            test = tests[j]
            outputs_mnist_n[j][r][i] = test(X, Y, subkey, seed)

outputs_mnist_n = jnp.array(outputs_mnist_n)   
power = jnp.array([[jnp.mean(outputs_mnist_n[j][r])for r in range(len(N_values))] for j in range(len(tests))])
np.save("results/mnist_n.npy", power)
np.save("results/mnist_n_x_axis.npy", N_values)

print("N_values :", N_values)
print("sample size :", N_values)
for t in range(len(tests)):
    print(" ")
    print(tests[t])
    print(outputs_mnist_n[t])

N_values : (200,)
sample size : (200,)
 
<function hsic_test at 0x7fbacc49f8b0>
[[0]]
